# Download

In [1]:
import requests
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm, trange
from bs4 import BeautifulSoup as bs
import os
import sqlalchemy as sa
import zipfile
import dask.dataframe as dd

In [8]:
# configuacoes
urlbase = "https://dadosabertos.rfb.gov.br/CNPJ/"
uri_sqlite ='sqlite:///base_receitaYYYYMMDD.db'
dirpath= "downloads/" # vai criar a pasta se ela não existir
if not os.path.exists(dirpath):
    os.makedirs(dirpath)

In [9]:
def mapp(fn,data, workers=8):
  tasks=[]
  out = []
  with ThreadPoolExecutor(max_workers=workers) as tex:
    tasks=[tex.submit(fn,i) for i in data]
  for task in as_completed(tasks):
    out.append(task.result())
  return out

In [10]:
def downloader(url):
    try:
        r = requests.get(url, stream=True)
        total = int(r.headers.get('content-length', 0))
        if "Content-Disposition" in r.headers.keys():
            fname = re.findall("filename=(.+)", r.headers["Content-Disposition"])[0]
        else:
            fname = url.split("/")[-1]
        with open(dirpath+fname, 'wb') as file, tqdm(desc=fname, total=total, unit='iB', unit_scale=True, unit_divisor=1024) as bar:
            for data in r.iter_content(chunk_size=1024):
                size = file.write(data)
                bar.update(size)
    except Exception as inst:
        print(inst)          # __str__ allows args to be printed directly,    
    finally:
        return [str(r.status_code), url]

In [11]:
filelist = list( filter( lambda s: s.find(".")>0,
                        [tag.attrs['href'] for tag in bs(requests.get(urlbase).content).find_all('a')]))

In [12]:
urllist = [urlbase+f for f in filelist]

In [13]:
res = mapp(downloader,urllist) if input("Tem certeza? S/N").upper() == "S" else "Cancelado"

Empresas0.zip:   0%|          | 0.00/240M [00:00<?, ?iB/s]

Empresas3.zip:   0%|          | 0.00/81.2M [00:00<?, ?iB/s]

Empresas5.zip:   0%|          | 0.00/93.3M [00:00<?, ?iB/s]

Cnaes.zip:   0%|          | 0.00/21.6k [00:00<?, ?iB/s]

Empresas2.zip:   0%|          | 0.00/75.3M [00:00<?, ?iB/s]

...

Socios5.zip:   0%|          | 0.00/46.9M [00:00<?, ?iB/s]

Socios6.zip:   0%|          | 0.00/46.8M [00:00<?, ?iB/s]

Socios7.zip:   0%|          | 0.00/46.8M [00:00<?, ?iB/s]

Socios8.zip:   0%|          | 0.00/46.8M [00:00<?, ?iB/s]

Socios9.zip:   0%|          | 0.00/46.8M [00:00<?, ?iB/s]

In [43]:
# se der erro em algum basta usar o downloader
# downloader(urllist[0])

In [14]:
res

[['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Paises.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Municipios.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Naturezas.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Estabelecimentos4.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Socios7.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Empresas0.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Empresas1.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Socios1.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Estabelecimentos7.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Estabelecimentos3.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Estabelecimentos6.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Empresas6.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Socios8.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Socios0.zip'],
 ['200', 'https://dadosabertos.rfb.gov.br/CNPJ/Estabelecimentos1.zip'],
 ['200', '

# Upload

## Funções

In [15]:
dirlist= os.listdir(dirpath)
engine = sa.create_engine(uri_sqlite)
engine.connect().close() #test

In [21]:
def upload(local, table, dcol):
    colunas=list(dcol.keys())
    with engine.connect() as c:
        c.execute(sa.text("DROP TABLE IF EXISTS \"" + table + "\""))
        block='default' if local.find('.zip') < 0 else None
        df = dd.read_csv(local, header=None, encoding='latin1', sep=";", decimal=",", names=colunas, dtype=dcol, blocksize=block)
    for n in trange(df.npartitions, desc=table):
        df.get_partition(n).compute().to_sql( name=table, con=engine, if_exists= 'append', chunksize=1000, index=False)
    with engine.connect() as c:
         c.execute(sa.text("CREATE INDEX IF NOT EXISTS idx_" + table + " ON \"" + table + "\" (\"" + df.columns[0] + "\")"))

In [22]:
def extract(flist,filtro):
    filenames=[]
    files=list(filter(lambda x: not x.find(filtro), dirlist))
    for filename in files:
            zip=zipfile.ZipFile(dirpath+filename)
            filenames.append(dirpath+zip.filelist[0].filename)
            zip.extractall(path=dirpath)
    return filenames

def remove(flist):
    for f in flist:
            os.remove(f)

## Começo do Upload

In [23]:
local=dirpath+'Municipios.zip'
dcol = {"CD_MUNICIPIO": "int","MUNICIPIO":"str"}
table='MUNICIPIO'
upload(local, table, dcol)

MUNICIPIO:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
local=dirpath+'Cnaes.zip'
dcol = {"CNAE": "int","NM_CNAE":"str"}
table='CNAE'
upload(local, table, dcol)

CNAE:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
local=dirpath+'Paises.zip'
dcol = {"CD_PAIS": "int","PAIS":"str"}
table='PAIS'
upload(local, table, dcol)

PAIS:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
local=dirpath+'Naturezas.zip'
dcol={"CD_NAT_JURIDICA": "int","NAT_JURIDICA":"str"}
table='NATJU'
upload(local, table, dcol)

NATJU:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
local=dirpath+'Motivos.zip'
dcols={"CD_MOTIVO_SIT_CADASTRO":"int","MOTIVO_SIT_CADASTRO":"str"}
table='MOTIVOSIT'
upload(local, table, dcol)

MOTIVOSIT:   0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
local=dirpath+'Qualificacoes.zip'
dcol={"CD_QUALS":"int","NM_QUALS":"str"}
table='QUALS'
upload(local, table, dcol)

QUALS:   0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
lista= extract(dirlist,"Simples")
#simples (cerca de 9min)
local=dirpath+'*.SIMPLES.*'
dcol={"CNPJ_BASICO":"int","OP_SIMPLES":"str","DT_OP_SIMPLES":"int","DT_EXC_SIMPLES":"int","OP_MEI":"str","DT_OP_MEI":"int","DT_EXC_MEI":"int"}
table='SIMPLES'
upload(local, table, dcol)
remove(lista)

SIMPLES:   0%|          | 0/34 [00:00<?, ?it/s]

In [30]:
%%time
lista= extract(dirlist,"Empre")
#empresa
dcol = {"CNPJ_BASICO":"int", "RAZAO_SOCIAL":"str", "NAT_JURIDICA":"int", "QUAL_RESP":"int", "CAPITAL_SOCIAL":"float",
        "PORTE_EMPRESA":"Int64", "ENTE_FED_RESP":"str"}
local=dirpath+'*.EMPRECSV'
table='EMPRESA'
upload(local, table, dcol)
remove(lista)

EMPRESA:   0%|          | 0/59 [00:00<?, ?it/s]

CPU times: total: 6min 4s
Wall time: 10min 21s


In [31]:
lista= extract(dirlist,"Socios")
#socios
dcol={"CNPJ_BASICO":"int","ID_TIPO_SOCIO":"int","NOME_OU_RAZAO_SOCIAL":"str","CNPJ_CPF":"str","QUALIF_SOCIO":"int",
         "DT_ENTRADA":"int","CD_PAIS":"Int64","REPR_LEGAL":"str","NM_REPR":"str","CD_QUALIF_REPR":"int","FAIXA_ETARIA":"int"}
local=dirpath+'*.SOCIOCSV'
table='SOCIO'
upload(local, table, dcol)
remove(lista)

SOCIO:   0%|          | 0/33 [00:00<?, ?it/s]

In [32]:
lista= extract(dirlist,"Estabelecimentos")
#estabelecimentos
dcol={"CNPJ_BASICO":"int", "CNPJ_ORDEM":"int", "CNPJ_DV":"int", "CD_MATRIZ_FILIAL":"int", "NM_FANTASIA":"str",
         "CD_SIT_CADASTRO":"int", "DT_SIT_CADASTRO":"int","MOTIVO_SIT_CADASTRO":"int","NM_CIDADE_EXT":"str", "CD_PAIS":"Int64",
         "DT_INI":"int", "CNAE_PRINCIPAL":"int","CNAE_SECUNDARIO":"str",
         "TIP_LOGRADOURO":"str","LOGRADOURO":"str","NUMERO":"str","COMPLEMENTO":"str","BAIRRO":"str","CEP":"str","UF":"str","MUNICIPIO":"int",
         "DDD_1":"str","TEL_1":"str","DDD_2":"str","TEL_2":"str","DDD_FAX":"str","FAX":"str","EMAIL":"str","SIT_ESP":"str","DT_SIT_ESP":"Int64"}
local=dirpath+'*.ESTABELE'
table='ESTABELE'
upload(local, table, dcol)
remove(lista)

ESTABELE:   0%|          | 0/198 [00:00<?, ?it/s]

In [33]:
remove([dirpath+f for f in filter(lambda x: x.find(".zip") > 0,dirlist)]) if input("Tem certeza? S/N").upper() == "S" else "Cancelado"

In [34]:
# Criar uma view para grafo societario
query = '''
CREATE view if not exists grapho as
SELECT
	SUBSTR("0000000" || CNPJ_BASICO,-8) as A,
	(case
		when CNPJ_CPF LIKE '***%' then (COALESCE(NOME_OU_RAZAO_SOCIAL,"") || " - " || CNPJ_CPF)
		when CNPJ_CPF is null then NOME_OU_RAZAO_SOCIAL
		else SUBSTR(CNPJ_CPF, 1, 8) end) as B
FROM SOCIO
union all
SELECT
	SUBSTR("0000000" || CNPJ_BASICO,-8) as A,
	(case
		when REPR_LEGAL LIKE '***%' then (NM_REPR || " - " || REPR_LEGAL)
		else NM_REPR end) as B
FROM SOCIO
WHERE NM_REPR is not null;
'''
engine.connect().execute(sa.text(query)).close()

## Testes